In [1]:
import math
import random
import numpy as np
from itertools import product

from qiskit import *

# Importing standard Qiskit libraries
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator

from qiskit.utils import QuantumInstance

from qiskit.opflow import I, X, Y, Z

from qiskit.opflow import StateFn
from qiskit.opflow import Gradient
from qiskit.opflow import CircuitSampler
from qiskit.opflow.primitive_ops import MatrixOp

from qiskit.quantum_info import Statevector

from qiskit.providers.aer.noise import NoiseModel
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
def createEqSuperposCS3():
    temp = QuantumCircuit(2)
    temp.ry(2*np.arctan(np.sqrt(2)), 1)
    temp.ch(1, 0)
    return temp

In [3]:
backend = Aer.get_backend('statevector_simulator')
result = execute(createEqSuperposCS3().reverse_bits(), backend).result().get_statevector()
print(np.real(result))

[5.77350269e-01 5.77350269e-01 8.32667268e-17 5.77350269e-01]


In [82]:
#Unitary to create state \rho
size = 4
state = QuantumCircuit(size)

#state.ry(2*np.arctan(np.sqrt(2)), 0)
#state.ch(0, 1)
#state.x(1)

state.h(0)
state.h(1)
state.cx(0, 2)
state.cx(1, 3)
state.draw()

┌───┐          
q_0: ┤ H ├──■───────
     ├───┤  │       
q_1: ┤ H ├──┼────■──
     └───┘┌─┴─┐  │  
q_2: ─────┤ X ├──┼──
          └───┘┌─┴─┐
q_3: ──────────┤ X ├
               └───┘

In [83]:
backend = Aer.get_backend('statevector_simulator')
result = execute(state.reverse_bits(), backend).result().get_statevector()
print(np.real(result))

[0.5 0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.5]


In [84]:
def createQuantumCircuit():
    circ = QuantumCircuit(6)
    #Create Quantum Circuit here
    
    superPosCircuit = createEqSuperposCS3()
    circ.append(superPosCircuit, [0, 1])
    circ.append(state, [2, 3, 4, 5])

    circ.ccx(0, 3, 2)
    circ.ccx(0, 2, 3)
    
    circ.ccx(1, 3, 2)
    circ.ccx(1, 2, 3)
    
    circ.append(superPosCircuit.inverse(), [0, 1])

    
    return circ

In [85]:
qCirc = createQuantumCircuit()
qCirc.draw()

┌────────────────┐                    ┌───────────────────┐
q_0: ┤0               ├──■────■────────────┤0                  ├
     │  circuit-49101 │  │    │            │  circuit-49101_dg │
q_1: ┤1               ├──┼────┼────■────■──┤1                  ├
     ├────────────────┤┌─┴─┐  │  ┌─┴─┐  │  └───────────────────┘
q_2: ┤0               ├┤ X ├──■──┤ X ├──■───────────────────────
     │                │└─┬─┘┌─┴─┐└─┬─┘┌─┴─┐                     
q_3: ┤1               ├──■──┤ X ├──■──┤ X ├─────────────────────
     │  circuit-49061 │     └───┘     └───┘                     
q_4: ┤2               ├─────────────────────────────────────────
     │                │                                         
q_5: ┤3               ├─────────────────────────────────────────
     └────────────────┘

In [86]:
#Create Hamiltonian here
hamiltonian = I^I^I^I^MatrixOp(np.array([[1, 0], [0, 0]]))^MatrixOp(np.array([[1, 0], [0, 0]]))

In [87]:
#use if noiseless
noiseless_backend = Aer.get_backend('statevector_simulator')
noiseless_q_instance = QuantumInstance(noiseless_backend)
noiseless_sampler = CircuitSampler(noiseless_q_instance)

In [88]:
'''
Pick a backend to use. We use the qasm_simulator for noisy simulation. 
'''
#Replace the provider with your personal credentials for hub, group and main
provider = IBMQ.get_provider(hub="ibm-q-research", group="louisiana-st-uni-1", project="main")

noisy_backend = Aer.get_backend("qasm_simulator")
device = provider.get_backend("ibmq_jakarta")
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device.properties())
noisy_q_instance = QuantumInstance(backend=noisy_backend, 
                           shots=8192, 
                           noise_model=noise_model, 
                           coupling_map=coupling_map,
                           measurement_error_mitigation_cls=CompleteMeasFitter,
                           cals_matrix_refresh_period=30)
noisy_sampler = CircuitSampler(noisy_q_instance)

In [89]:
def costf(noisy = False):
    expectation = StateFn(hamiltonian, is_measurement=True).compose(StateFn(qCirc))
    
    if noisy:
        result = noisy_sampler.convert(expectation).eval()  
    else:
        result = noiseless_sampler.convert(expectation).eval()  
    
    return np.real(result)

In [94]:
print("Initial Cost : ", costf(True))

Initial Cost :  0.469815182717453
